In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Github/sports-article-classification/

%pip install -r requirements_colab.txt
%pip install transformers
%pip install datasets
%pip install slackclient
%pip install doccano-client
%pip install wandb
%pip install scikit-multilearn
%cd src/modelling

%cd ../..
%pip install -e .


/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/Github/sports-article-classification (from -r requirements_colab.txt (line 14))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.0 MB/s eta 0

In [6]:
%cd Github/sports-article-classification/
%pip install -e .


/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/Github/sports-article-classification
  Preparing metadata (setup.py) ... done
  Attempting uninstall: Sports-article-classification-packages
    Found existing installation: Sports-article-classification-packages 0.1.0
    Can't uninstall 'Sports-article-classification-packages'. No files were found to uninstall.
  Running setup.py develop for Sports-article-classification-packages


In [7]:
from src.data.preprocessing import DataPreprocessorHelland
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
#import ray.data
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from src.slack_alert.sofus_alert import sofus_alert
from huggingface_hub import login
import configparser
from src.utils import get_project_root
from src.data.dataset import get_training_data
#from ray.tune import JupyterNotebookReporter
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from transformers import EvalPrediction
import torch
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification
import wandb

In [8]:
%cd src/modelling

/content/gdrive/MyDrive/Github/sports-article-classification/src/modelling


In [9]:
data = get_training_data()
data = data.loc[:, ['text', 'label']]
data_preprocessor = DataPreprocessorHelland(data['text'])
data_preprocessor.make_lower_cased()
data_preprocessor.remove_extra_spaces()
data['text'] = data_preprocessor.text_series
one_hot = MultiLabelBinarizer()
one_hot_label = one_hot.fit_transform(data['label'])
one_hot_label_list = [list(map(float, x)) for x in one_hot_label]
data['label'] = pd.Series(list(one_hot_label_list))


test_size = 0.2
X = data['text']
y = np.stack(data['label'])
stratifier = IterativeStratification(n_splits=2, order=3, sample_distribution_per_fold=[test_size, 1.0-test_size])
train_indexes, test_indexes = next(stratifier.split(X, y))

y = pd.Series(list(y), name='label')
train_split = pd.concat([X[train_indexes], y[train_indexes]], axis=1)
validation_split = pd.concat([X[test_indexes], y[test_indexes]], axis=1)

train_dataset = Dataset.from_pandas(train_split)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = Dataset.from_pandas(validation_split)
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])
split_dict = {'train': train_dataset, 'validation': validation_dataset}
datasets = DatasetDict(split_dict)

In [10]:
model_checkpoint = 'NbAiLab/nb-bert-large'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, model_max_length=256) #256

In [11]:
def preprocess_function(examples):

    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256) #256

encoded_dataset = datasets.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    #f1_classwise = f1_score(y_true=y_true, y_pred=y_pred, average=None)
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    score_report = classification_report(y_true,y_pred, output_dict=True, target_names=one_hot.classes_)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'score_report': score_report}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [13]:
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=22, problem_type="multi_label_classification")

Some weights of the model checkpoint at NbAiLab/nb-bert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [14]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=22, problem_type="multi_label_classification", return_dict=True)

In [15]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token_hf = config['huggingface']['TOKEN']

login(token=token_hf)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 393 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50000, 1024)
bert.embeddings.position_embeddings.weight               (512, 1024)
bert.embeddings.token_type_embeddings.weight               (2, 1024)
bert.embeddings.LayerNorm.weight                             (1024,)
bert.embeddings.LayerNorm.bias                               (1024,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.query.bias               (1024,)
bert.encoder.layer.0.attention.self.key.weight          (1024, 1024)
bert.encoder.layer.0.attention.self.key.bias                 (1024,)
bert.encoder.layer.0.attention.self.value.weight        (1024, 1024)
bert.encoder.layer.0.attention.self.value.bias               (1024,)
bert.encoder.layer.0.attention.output.dense.weight      (1024, 1024)
bert.encoder.layer.0.attention.output.dense.bias             (

In [17]:
metric_name = 'accuracy'
model_name = model_checkpoint.split('/')[-1]
task = 'multi_label_classification'

args = TrainingArguments(
    output_dir = f'final-{model_name}-finetuned-{task}',
    evaluation_strategy = 'epoch',
    save_strategy = 'no',
    seed = 100,
    run_name='sports-article-classification',
#    learning_rate=2e-5,
#    per_device_train_batch_size=batch_size,
#    per_device_eval_batch_size=batch_size,
#    num_train_epochs=5,
#    weight_decay=0.01,
#   load_best_model_at_end=False,
    #metric_for_best_model=metric_name,
    #push_to_hub=True
)

In [18]:
validation_key = "validation"
trainer = Trainer(
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
   

In [19]:
def wandb_hp_space(trial):
    return {
        'method': 'bayes',
        'metric': {'name': 'objective', 'goal': 'maximize'},
        'parameters': {
            'epoch': {'values': [3, 4, 5]},
            'learning_rate': {'distribution': 'uniform', 'min': 1e-6, 'max': 1e-4},
            'per_device_train_batch_size': {'values': [16, 32]},
            'warmup_ratio': {'distribution': 'uniform', 'min': 0, 'max': 0.5},
            'weight_decay': {'distribution': 'uniform', 'min': 0, 'max': 0.3}
        }}

In [ ]:
def spesific_hp_space(trial):
    return {
          'method': 'bayes',
          'metric': {'name': 'objective', 'goal': 'maximize'},
          'parameters': {
              'epoch': {'values': [3]},
              'learning_rate': {'values': [0.0000995095884931505]},
              'per_device_train_batch_size': {'values': [16]},
              'warmup_ratio': {'values': [0]},
              'weight_decay': {'values': [0]}
          }}

In [20]:
def optimize_objective(metrics):

    #objective_metrics = ['weighted avg', '']
    objective = metrics['eval_score_report']['weighted avg']['f1-score']


    #objective_metrics = ['eval_f1_macro', 'eval_roc_auc', 'eval_accuracy']
    #objective = dict(filter(lambda item: item[0] in objective_metrics, metrics.items()))
    return objective

In [21]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token_wandb = config['wandb']['TOKEN']
wandb.login(key=token_wandb)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
bestrun = trainer.hyperparameter_search(
    direction='maximize', 
    backend='wandb', 
    n_trials=30, # number of trials
    hp_space=wandb_hp_space,
    compute_objective=optimize_objective,
    **{'project':'sports-article-classification-final'})

wandb sweep id - ywa10pac


Create sweep with ID: ywa10pac
Sweep URL: https://wandb.ai/eirik-helland/sports-article-classification-final/sweeps/ywa10pac


wandb: Agent Starting Run: n5xb7f5s with config:
wandb: 	epoch: 5
wandb: 	learning_rate: 5.795792678706725e-05
wandb: 	per_device_train_batch_size: 16
wandb: 	warmup_ratio: 0.49971815426623406
wandb: 	weight_decay: 0.15748609326170407
wandb: Currently logged in as: eirik-helland. Use `wandb login --relogin` to force relogin


Trying to set epoch in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
W&B Sweep parameters: {'epoch': 5, 'learning_rate': 5.795792678706725e-05, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.49971815426623406, 'weight_decay': 0.15748609326170407, '_wandb': {}, 'assignments': {}, 'metric': 'eval/loss'}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--NbAiLab--nb-bert-large/snapshots/27e8180855f0de03688958c88a2e5702bfbf0bfd/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Roc Auc,Accuracy,Score Report
1,No log,0.159447,0.551509,0.112504,0.701269,0.320802,"{'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'Chance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'Description': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 42}, 'Garbage': {'precision': 0.9583333333333334, 'recall': 0.5111111111111111, 'f1-score': 0.6666666666666666, 'support': 45}, 'Goal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 35}, 'Injury': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, 'Link': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Next game': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11}, 'Odds': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'Opinion': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, 'Quote': {'precision': 0.9782608695652174, 'recall': 1.0, 'f1-score': 0.989010989010989, 'support': 90}, 'Rumour': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Set-piece': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Statement': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 78}, 'Statistics': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Storytelling': {'precision': 0.7755102040816326, 'recall': 0.8685714285714285, 'f1-score': 0.8194070080862533, 'support': 175}, 'Substitution': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'Summary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 46}, 'Table': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 14}, 'Transfer': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11}, 'VAR': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'micro avg': {'precision': 0.8493589743589743, 'recall': 0.4083204930662558, 'f1-score': 0.5515088449531738, 'support': 649}, 'macro avg': {'precision': 0.12327747304455379, 'recall': 0.10816738816738815, 'f1-score': 0.11250384835290496, 'support': 649}, 'weighted avg': {'precision': 0.411221516140455, 'recall': 0.4083204930662558, 'f1-score': 0.4043254474978171, 'support': 649}, 'samples avg': {'precision': 0.6641604010025063, 'recall': 0.4851712614870509, 'f1-score': 0.5423558897243108, 'support': 649}}"
2,No log,0.136729,0.617934,0.173817,0.740531,0.348371,"{'Booking': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}, 'Chance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 8}, 'Commentary': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'Description': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 42}, 'Garbage': {'precision': 0.9722222222222222, 'recall': 0.7777777777777778, 'f1-score': 0.8641975308641976, 'support': 45}, 'Goal': {'precision': 1.0, 'recall': 0.2571428571428571, 'f1-score': 0.40909090909090906, 'support': 35}, 'Injury': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}, 'Link': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Next game': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 11}, 'Odds': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'Opinion': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, 'Quote': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 90}, 'Rumour': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'Set-piece': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Statement': {'precision': 0.5, 'recall': 0.02564102564102564, 'f1-score': 0.048780487804878044, 'support': 78}, 'Statistics': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Storytelling': {'precision': 0.7731958762886598, 'recall': 0.8571428571428571, 'f1-score': 0.8130081300

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 399
  Batch size = 8
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'Booking'

eval/accuracy,▁▄█
eval/f1_macro,▁▄█
eval/f1_micro,▁▅█
eval/loss,█▄▁
eval/roc_auc,▁▅█
eval/runtime,▂█▁
eval/samples_per_second,▇▁█
eval/steps_per_second,▇▁█
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁


In [ ]:
sofus_alert()

In [ ]:
from google.colab import runtime
runtime.unassign()